In [1]:
import regex as re
import pandas as pd

In [2]:
f=open('./apache_15k.log')
lines=f.readlines()

In [3]:
split_pt=re.compile(r'([\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:,\.](?=\s)|(?<=\b[a-zA-Z]+):(?=\d+|[a-zA-Z]+\b))')
hm_pt=re.compile(r'(\[.*?\]\s\[.*?\])(.*)')

In [4]:
header=[]
msg=[]
for line in lines:
    h,m=hm_pt.match(line).groups()
    
    header.append(h)
    msg.append(m)

In [5]:
print(header[0])
print(msg[0])

[Fri Feb 24 17:07:21 2006] [error]
 [client 85.214.20.161] File does not exist: /var/www/html/cvs


In [6]:
header_tokens=[]
for h in header:
    tokens=[token for token in split_pt.split(h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [7]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in split_pt.split(m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [8]:
pd.DataFrame({'header_tokens':header_tokens,'msg_tokens':msg_tokens}).to_csv('tokens.csv')

In [9]:
header_anno=[]
for tokens in header_tokens:
    anno=['O']*len(tokens)
    anno[1:6],anno[7],anno[13]=['B-DATE']+['I-DATE']*4,'B-TIME','B-LVL'

    header_anno.append(anno)

In [10]:
list(zip(header_tokens[0],header_anno[0]))

[('[', 'O'),
 ('Fri', 'B-DATE'),
 (' ', 'I-DATE'),
 ('Feb', 'I-DATE'),
 (' ', 'I-DATE'),
 ('24', 'I-DATE'),
 (' ', 'O'),
 ('17:07:21', 'B-TIME'),
 (' ', 'O'),
 ('2006', 'O'),
 (']', 'O'),
 (' ', 'O'),
 ('[', 'O'),
 ('error', 'B-LVL'),
 (']', 'O')]

In [11]:
msg_anno=[]
for tokens in msg_tokens:
    anno=['<O>']*len(tokens)
    msg_anno.append(anno)

In [12]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/'
re_ip=r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [13]:
def sub_tag(ntokens,ntags, patt_tag_list):

    def _sub_tag(ntokens,ntags, patt, target_tag):
        for tokens,tags in zip(ntokens,ntags):
            for i in range(len(tokens)):
                if re.search(patt,tokens[i]):
                    tags[i]=target_tag

    for patt,tag in patt_tag_list:
        _sub_tag(ntokens,ntags,patt,tag)

In [14]:
sub_tag(msg_tokens,msg_anno,
    [(re_cls,'B-CLS'),
    (re_func,'B-FUNC'),
    (re_path,'B-PATH'),
    (re_ip,'B-IP'),
    (re_host,'B-HOST'),
    (re_url,'B-URL')])

In [15]:
list(zip(msg_tokens[0],msg_anno[0]))

[(' ', '<O>'),
 ('[', '<O>'),
 ('client', '<O>'),
 (' ', '<O>'),
 ('85.214.20.161', 'B-IP'),
 (']', '<O>'),
 (' ', '<O>'),
 ('File', '<O>'),
 (' ', '<O>'),
 ('does', '<O>'),
 (' ', '<O>'),
 ('not', '<O>'),
 (' ', '<O>'),
 ('exist', '<O>'),
 (':', '<O>'),
 (' ', '<O>'),
 ('/var/www/html/cvs', 'B-PATH')]

In [16]:
with open('./header_check.txt','w') as f:
    for tokens,tags in zip(header_tokens,header_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [17]:
with open('./msg_check.txt','w') as f:
    for tokens,tags in zip(msg_tokens,msg_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [18]:
df=pd.DataFrame({
    'tokens':[h+m for h,m in zip(header_tokens,msg_tokens)],
    'tags':[h+m for h,m in zip(header_anno,msg_anno)],
    'raw_log':[h+m for h,m in zip(header,msg)]
    })
df.to_csv('./apache_ds.csv')